In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Load the training data
train_path = '/home/data/train.json'

# Read the JSON file
train_data = []
with open(train_path, 'r') as f:
    for line in f:
        train_data.append(json.loads(line))

# Convert to DataFrame
df_train = pd.DataFrame(train_data)
print(f"Training data shape: {df_train.shape}")
print(f"Columns: {df_train.columns.tolist()}")
print("\nFirst few rows:")
df_train.head()